<a href="https://colab.research.google.com/github/VMadhav007/ML/blob/main/Langchain_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install langchain_huggingface;langchain_community.vectorstores;langchain_community.document_loaders;langchain_text_splitters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [6]:
!pip install langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [7]:

#from langchain.chat_models import init_chat_model
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain

In [8]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 5.2 MB/s eta 0:00:00


In [18]:
PDF_PATH=r"/content/2.pdf"
FAISS_INDEX_PATH = "faiss_index"  # Directory to save FAISS index

#Load the PDFs
loader = PyPDFLoader(PDF_PATH)
docs=loader.load()
print(f"The Length of the Documents is: {len(docs)}")
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

The Length of the Documents is: 19
OPERATING SYSTEMS
UE22CS242B
File Management
Suresh Jamadagni
Department of Computer Science

{'producer': 'Online2PDF.com', 'creator': 'Online2PDF.com', 'creationdate': '2025-04-10T23:01:50+02:00', 'source': '/content/2.pdf', 'total_pages': 19, 'page': 0, 'page_label': '1'}


In [19]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)
print(len(all_splits))


19


In [20]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 768

[0.03649519756436348, -0.015482468530535698, 0.0030441097915172577, -0.005028152372688055, -0.013247640803456306, 0.02611396089196205, 0.04672618210315704, -0.007720525376498699, 0.004780058283358812, -0.015757201239466667]


In [21]:
from langchain_community.vectorstores import FAISS

In [22]:
!pip install faiss-cpu


In [23]:

vector_store = FAISS.from_documents(all_splits, embeddings)
print("FAISS Vector Store is created")


print("*** Using Similarity Search ***")
#Use the Vector Store/DB and use a Similarity Search to get information from the Vector DB
results = vector_store.similarity_search(
    "What is the single level directory"
)
print(results[0])

print("*** Using Similarity Search with Score ***")
results = vector_store.similarity_search_with_score("What is the single level directory")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

print("*** Return the Embedding Documents that were retrieved for the Query")
embedding = embeddings.embed_query("What is the single level directory")
results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

FAISS Vector Store is created
*** Using Similarity Search ***
page_content='Single-Level Directory
OPERATING SYSTEMS
● A single directory for all users
● Naming problem
● Grouping problem' metadata={'producer': 'Online2PDF.com', 'creator': 'Online2PDF.com', 'creationdate': '2025-04-10T23:01:50+02:00', 'source': '/content/2.pdf', 'total_pages': 19, 'page': 9, 'page_label': '10', 'start_index': 0}
*** Using Similarity Search with Score ***
Score: 0.7426968216896057

page_content='Single-Level Directory
OPERATING SYSTEMS
● A single directory for all users
● Naming problem
● Grouping problem' metadata={'producer': 'Online2PDF.com', 'creator': 'Online2PDF.com', 'creationdate': '2025-04-10T23:01:50+02:00', 'source': '/content/2.pdf', 'total_pages': 19, 'page': 9, 'page_label': '10', 'start_index': 0}
*** Return the Embedding Documents that were retrieved for the Query
page_content='Single-Level Directory
OPERATING SYSTEMS
● A single directory for all users
● Naming problem
● Grouping problem

In [25]:
print("\n\n*** Retrievers - Using vector store as a Retriever ***")
query = "What is the single level directory"
similarity_retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2},
)
similarity_results = similarity_retriever.get_relevant_documents(query)
print("\n=== SIMILARITY SEARCH RESULTS ===")
for i, doc in enumerate(similarity_results):
    print(f"--- Chunk {i+1} ---")
    print(doc.page_content[:500])  # Show first 300 characters
    print()

mmr_retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 2},
)
mmr_results = mmr_retriever.get_relevant_documents(query)
print("\n=== MMR SEARCH RESULTS ===")
for i, doc in enumerate(mmr_results):
    print(f"--- Chunk {i+1} ---")
    print(doc.page_content[:500])
    print()



*** Retrievers - Using vector store as a Retriever ***

=== SIMILARITY SEARCH RESULTS ===
--- Chunk 1 ---
Single-Level Directory
OPERATING SYSTEMS
● A single directory for all users
● Naming problem
● Grouping problem

--- Chunk 2 ---
Directory Structure
OPERATING SYSTEMS
● A collection of nodes containing information about all files
F 
1
F 
2 F 
3
F 
4 F 
n
Directory
Files
Both the directory structure and the files reside on disk


=== MMR SEARCH RESULTS ===
--- Chunk 1 ---
Single-Level Directory
OPERATING SYSTEMS
● A single directory for all users
● Naming problem
● Grouping problem

--- Chunk 2 ---
Tree-Structured Directories (Cont.)
OPERATING SYSTEMS
● Absolute or relative path name
● Creating a new file is done in current directory
● Delete a file
rm <file-name>
● Creating a new subdirectory is done in current directory
mkdir <dir-name>
❑ Example:  if in current directory   /mail
mkdir count
Deleting “mail” ⇒ deleting the entire subtree rooted by “mail”

